In [91]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd

## input variable

In [99]:
View_Column_name = "PrevContext_NextContext_PrevTarget_NextTarget_ShapeTarget"
DATE = input()
DATE

'2021-01-14/22-12-22/projector/in_test/199'

## load data

In [100]:
# load csv
df = pd.read_csv("../outputs/{}/tensor.csv".format(DATE), header=None, delimiter="\t")
df_label = pd.read_csv("../outputs/{}/metadata.tsv".format(DATE), delimiter="\t", index_col=0)

In [101]:
data = pd.concat([df_label, df], axis=1)
del df, df_label

## def loss

In [102]:
import scipy.spatial.distance
def cossim(x, y):
    return scipy.spatial.distance.cosine(x, y)

def mean_cossim(X=context_x, Y=target_x):
    tmp = 0.
    for i in range(context_x.shape[0]):
        x = X[i, :]
        y = Y[i, :]
        tmp += cossim(x, y)
    return 1-tmp/context_x.shape[0]

## select vector

In [103]:
vector_dict = {
    "prev_context": None,
    "next_context": None,
    "prev_target": None,
    "next_target": None,
    "now_target": None,
}
for i, name in enumerate(vector_dict.keys()):
     vector_dict[name] = np.array(data.loc[data[View_Column_name]==i].iloc[:, 2:])

## Calculate

In [104]:
tmp = []
for i, (k, v) in enumerate(vector_dict.items()):
    for j, (k_, v_) in enumerate(vector_dict.items()):
        if i<j and k != k_:
            cossim_value = mean_cossim(v, v_)
            print("{} VS {} \t\t {:.10f}".format(k, k_, cossim_value))
            tmp.append([k, k_, cossim_value, 1-cossim_value])

prev_context VS next_context 		 0.9705396609
prev_context VS prev_target 		 0.9995694546
prev_context VS next_target 		 0.9722395823
prev_context VS now_target 		 0.9919900052
next_context VS prev_target 		 0.9706825441
next_context VS next_target 		 0.9993673065
next_context VS now_target 		 0.9870348768
prev_target VS next_target 		 0.9723826613
prev_target VS now_target 		 0.9920406493
next_target VS now_target 		 0.9878624182


In [105]:
# 20k samples
df = pd.DataFrame(tmp, columns=["vector A", "vector B", "cossim", "cossim_loss"])
df.sort_values("cossim", ascending=False)

,vector A,vector B,cossim,cossim_loss
1,prev_context,prev_target,0.999569,0.000431
5,next_context,next_target,0.999367,0.000633
8,prev_target,now_target,0.992041,0.007959
3,prev_context,now_target,0.991990,0.008010
9,next_target,now_target,0.987862,0.012138
6,next_context,now_target,0.987035,0.012965
7,prev_target,next_target,0.972383,0.027617
2,prev_context,next_target,0.972240,0.027760
4,next_context,prev_target,0.970683,0.029317
0,prev_context,next_context,0.970540,0.029460
